# Práctica 1

Integrantes:

Hernández Sosa Javier Arturo

Hernández Jiménez Valeria

Meneses Orellana Rodrigo Alfredo

Becerril Dávila Tania Edith 

Saavedra Villavicencio Jesús

Baltazar Hernandez Jesus Armando

Flores Hernández Efraín Ismael


## Importar

In [1]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
file = '/home/ef/Documents/Diplomado/data/'

In [2]:
ratings = pd.read_csv(file + 'ratings.csv')
tags = pd.read_csv(file + 'tags.csv')
movies = pd.read_csv(file + 'movies.csv')

## Ingeniería de variables

In [3]:
aux = []
for x in movies['title']:
    try:
        a = int(x.strip()[-5:][:-1])
    except:
        aux.append(np.nan)
    else:
        aux.append(a)
        
movies['Año'] = aux
movies['count_genres'] = movies['genres'].map(lambda x: len(x.split('|')))
movies[['gen1',
        'gen2',
        'gen3']] = movies['genres'].str.split('|',
                                              expand = True).iloc[:,:3].fillna('No_aplica')
movies.drop(columns = ['genres'], inplace = True)

In [4]:
import re
import nltk
import unicodedata
from nltk.corpus import stopwords

texto = 'title'
nltk.download('stopwords')
stop_words = stopwords.words('english')
def clean_text(text, pattern='[^a-zA-Z]'):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, ' ', cleaned_text.decode('utf-8'), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().split())
    return cleaned_text  
total = []
for x in [n.split() for n in [clean_text(x) for x in movies[texto]]]:
    aux = []
    for word in x:
        aux.append(word)
    total.append(aux)
movies[f'{texto}_limpio'] = [' '.join(x) for x in total]
movies[f'{texto}_long'] = movies[texto].str.len()
movies[f'{texto}_n_words'] = movies[texto].str.split().str.len()
movies[f'{texto}_n_letters'] = movies[texto].map(lambda x:sum(map(str.isalpha, x)))

[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from datetime import datetime
ratings['timestamp'] = [datetime.fromtimestamp(x) for x in ratings['timestamp']]
ratings['timestamp'] = ratings['timestamp'].dt.year
ratings.rename(columns = {'timestamp':'Año'}, inplace = True)
ratings = ratings.groupby("movieId").agg({'userId':'count',
                                          'rating':['median',
                                                    'mean',
                                                    'min',
                                                    'max'],
                                          'Año':['median',
                                                 'mean',
                                                 'min',
                                                 'max']})
ratings.columns = [x+"_"+y for x,y in ratings.columns]
ratings.reset_index(inplace = True)

In [6]:
tags['timestamp'] = [datetime.fromtimestamp(x) for x in tags['timestamp']]
tags['timestamp'] = tags['timestamp'].dt.year
tags.rename(columns = {'timestamp':'Año'}, inplace = True)

In [7]:
mensaje = 'tag'
nltk.download('stopwords')
stop_words = stopwords.words('english')
def clean_text(text, pattern='[^a-zA-Z]'):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, ' ', cleaned_text.decode('utf-8'), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().split())
    return cleaned_text  
texto = []
for x in [n.split() for n in [clean_text(x) for x in tags[mensaje]]]:
    aux = []
    for word in x:
        aux.append(word)
    texto.append(aux)
tags[f'{mensaje}_limpio'] = [' '.join(x) for x in texto]
tags[f'{mensaje}_long'] = tags[mensaje].str.len()
tags[f'{mensaje}_n_words'] = tags[mensaje].str.split().str.len()
tags[f'{mensaje}_n_letters'] = tags[mensaje].map(lambda x:sum(map(str.isalpha, x)))

[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = TfidfVectorizer(ngram_range = (1, 1), 
                     min_df = 10, 
                     max_features = 50)
texto = 'tag_limpio'
cv.fit(tags[texto])
words = cv.get_feature_names()
tags = tags.join(pd.DataFrame(data = cv.transform(tags[texto]).todense(),
                              columns = words))

In [9]:
import itertools

num_tag = list(tags.iloc[:,5:8].columns)
funciones = itertools.repeat(['median','mean','min','max'],len(num_tag))
funciones = dict(zip(num_tag,funciones))
funciones.update(dict(itertools.zip_longest(words,itertools.repeat('sum',len(words)))))

tags = tags.groupby('movieId').agg(funciones)
tags.columns = [x+'_'+y for x,y in tags.columns]
tags.reset_index(inplace = True)

## Unión

In [10]:
df = movies.merge(ratings,
                  on = 'movieId', 
                  how = 'left').dropna().merge(tags, 
                                               on = 'movieId', 
                                               how = 'left')
df.fillna(0, inplace = True)
df.drop(columns = ['movieId', 'title_limpio'], inplace = True)
df.set_index('title', inplace = True)

In [11]:
df.sample(4)

,Año,count_genres,gen1,gen2,gen3,title_long,title_n_words,title_n_letters,userId_count,rating_median,rating_mean,rating_min,rating_max,Año_median,Año_mean,Año_min,Año_max,tag_long_median,tag_long_mean,tag_long_min,tag_long_max,tag_n_words_median,tag_n_words_mean,tag_n_words_min,tag_n_words_max,tag_n_letters_median,tag_n_letters_mean,tag_n_letters_min,tag_n_letters_max,action_sum,and_sum,animation_sum,appealing_sum,atmospheric_sum,bad_sum,black_sum,business_sum,classic_sum,comedy_sum,crime_sum,dark_sum,disney_sum,ending_sum,fi_sum,funny_sum,good_sum,great_sum,high_sum,humor_sum,illness_sum,in_sum,king_sum,mental_sum,movie_sum,music_sum,netflix_sum,of_sum,on_sum,plot_sum,politics_sum,provoking_sum,psychology_sum,queue_sum,quirky_sum,religion_sum,school_sum,sci_sum,soundtrack_sum,space_sum,superhero_sum,surreal_sum,suspense_sum,thought_sum,time_sum,travel_sum,twist_sum,violence_sum,visually_sum,war_sum
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Field of Dreams (1989),1989.00,3,Children,Drama,Fantasy,22,4,13,56.00,3.75,3.52,0.50,5.00,2004.00,2005.64,1996.00,2018.00,8.00,8.00,8.00,8.00,1.00,1.00,1.00,1.00,8.00,8.00,8.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Herbie: Fully Loaded (2005),2005.00,3,Adventure,Comedy,Romance,27,4,17,3.00,3.00,3.17,2.00,4.50,2015.00,2014.00,2009.00,2018.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Unforgiven (1992),1992.00,2,Drama,Western,No_aplica,17,2,10,45.00,4.00,4.11,2.00,5.00,2004.00,2005.38,1996.00,2018.00,7.00,7.00,7.00,7.00,1.00,1.00,1.00,1.00,7.00,7.00,7.00,7.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Yearling, The (1946)",1946.00,2,Children,Drama,No_aplica,20,3,11,1.00,4.00,4.00,4.00,4.00,2004.00,2004.00,2004.00,2004.00,12.00,12.00,12.00,12.00,2.00,2.00,2.00,2.00,11.00,11.00,11.00,11.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Distancias

In [12]:
num = [x for x in df.describe().columns]
cat = [x for x in df.columns if x not in num]
df = df[cat + num].copy()

In [13]:
## Dummies para categóricas
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

## Escala para numéricas
from sklearn.preprocessing import MinMaxScaler
mm_x = MinMaxScaler()

## Se aplicará transformación para cada tipo de columnas
from sklearn.compose import ColumnTransformer
prep = ColumnTransformer(transformers=[('OHE', ohe, cat),
                                       ('Scale', mm_x, num)])

In [14]:
df = pd.DataFrame(prep.fit_transform(df), index = df.index)

df.sample(4)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
People Will Talk (1951),0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.42,0.11,0.10,0.08,0.12,0.00,0.67,0.67,0.67,0.67,0.36,0.36,0.36,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
anohana: The Flower We Saw That Day - The Movie (2013),0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.96,0.11,0.31,0.35,0.32,0.00,0.56,0.56,0.56,0.56,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Unsinkable Molly Brown, The (1964)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.53,0.00,0.17,0.12,0.20,0.00,0.72,0.72,0.67,0.78,0.30,0.30,0.18,0.41,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Nature Calls (2012),0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.95,0.00,0.07,0.04,0.10,0.00,0.22,0.22,0.22,0.22,0.82,0.82,0.82,0.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [15]:
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('manhattan')
resultado = pd.DataFrame(dist.pairwise(df), index = df.index, columns = df.index)

## Validación

In [16]:
pelis = ['Toy Story (1995)',
'Harry Potter and the Prisoner of Azkaban (2004)',
'Avengers, The (1998)',
'Alice in Wonderland (1933)',
'Titanic (1997)',
'Shrek (2001)',
'Hercules (1997)',
'Finding Nemo (2003)',
'La La Land (2016)',
'Tomb Raider (2018)']

In [17]:
for peli in pelis:
    display(resultado.loc[:,peli].sort_values()[3:8].reset_index())

,title,Toy Story (1995)
0,Shrek 2 (2004),1.65
1,Hercules (1997),1.70
2,"Monsters, Inc. (2001)",1.77
3,"Secret of NIMH, The (1982)",1.87
4,Space Jam (1996),2.00


,title,Harry Potter and the Prisoner of Azkaban (2004)
0,Harry Potter and the Goblet of Fire (2005),2.41
1,Alice in Wonderland (2010),2.79
2,Harry Potter and the Chamber of Secrets (2002),2.90
3,"Lord of the Rings: The Two Towers, The (2002)",3.15
4,Jack the Giant Slayer (2013),3.56


,title,"Avengers, The (1998)"
0,Vertical Limit (2000),0.81
1,"Quest, The (1996)",0.88
2,"Phantom, The (1996)",1.27
3,Lara Croft: Tomb Raider (2001),1.66
4,"Ghost and the Darkness, The (1996)",1.69


,title,Alice in Wonderland (1933)
0,Sapphire Blue (2014),2.02
1,Ruby Red (2013),2.09
2,"Water Horse: Legend of the Deep, The (2007)",2.09
3,Race to Witch Mountain (2009),2.13
4,Bridge to Terabithia (2007),2.55


,title,Titanic (1997)
0,"Beautiful Mind, A (2001)",1.33
1,"Virgin Suicides, The (1999)",1.53
2,Hoosiers (a.k.a. Best Shot) (1986),1.62
3,Monster's Ball (2001),1.63
4,Before Sunrise (1995),1.68


,title,Shrek (2001)
0,Finding Nemo (2003),1.70
1,Toy Story (1995),2.00
2,"Monsters, Inc. (2001)",2.01
3,WALL·E (2008),2.20
4,"Bug's Life, A (1998)",2.21


,title,Hercules (1997)
0,Anastasia (1997),0.84
1,Mulan (1998),0.85
2,Tarzan (1999),0.93
3,"Monsters, Inc. (2001)",0.97
4,"Rescuers, The (1977)",1.00


,title,Finding Nemo (2003)
0,"Monsters, Inc. (2001)",1.78
1,"Bug's Life, A (1998)",1.85
2,Ice Age (2002),1.86
3,Aladdin (1992),1.88
4,Toy Story (1995),2.02


,title,La La Land (2016)
0,Vicky Cristina Barcelona (2008),2.51
1,Juno (2007),2.98
2,"Good Year, A (2006)",2.99
3,Zack and Miri Make a Porno (2008),3.11
4,"Room, The (2003)",3.31


,title,Tomb Raider (2018)
0,Kong: Skull Island (2017),2.90
1,Wonder Woman (2017),2.96
2,Fantastic Four (2015),3.01
3,Warcraft (2016),3.07
4,X-Men: Apocalypse (2016),3.09


In [18]:
def recomenda(n = 5):
    while True:
        try:
            peli = input('Qué peli te gusta?: ')
            resultado[peli]
        except:
            print(f'La peli "{peli}" no está en el catálogo, intenta de nuevo!\n')
            continue
        else:
            print(f'Estas {n} pelis podrían gustarte también: ')
            display(resultado[peli].sort_values()[3:3+n].reset_index())
            break

In [19]:
recomenda()

Qué peli te gusta?: Inception (2010)
Estas 5 pelis podrían gustarte también: 


,title,Inception (2010)
0,"Dark Knight, The (2008)",8.15
1,3:10 to Yuma (2007),8.43
2,Taken (2008),8.61
3,Scarface (1983),8.61
4,Insomnia (2002),8.90


In [20]:
recomenda(7)

Qué peli te gusta?: Star Wars
La peli "Star Wars" no está en el catálogo, intenta de nuevo!

Qué peli te gusta?: Star Wars: Episode III - Revenge of the Sith (2005)
Estas 7 pelis podrían gustarte también: 


,title,Star Wars: Episode III - Revenge of the Sith (2005)
0,Spider-Man 2 (2004),1.92
1,Hulk (2003),2.03
2,X2: X-Men United (2003),2.09
3,Star Wars: Episode II - Attack of the Clones (...,2.17
4,Jurassic Park III (2001),2.30
5,Star Wars: Episode VI - Return of the Jedi (1983),2.36
6,"Lost World: Jurassic Park, The (1997)",2.46
